<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Building_GNNmodel_with_PyG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#!pip install torch-geometric

# Importing Data and Libraries

In [11]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
cBio = pd.read_csv('/content/drive/MyDrive/gsoc data/cBio.csv')

In [14]:
cBio = cBio.set_index('Sample Identifier')
cBio.index.name = None
cBio.head(2)

,Overall Survival (Months),NUDT18,SHC3,NCRNA00112,FAM49B,FMO6P,BAT2L2,SIRT1,MSLNL,ATG4A,...,SCARNA6,SNORD77,TLR10,GKN2,PCDHGB4,ZMAT5,AXIN2,LONRF1,KLHL10,C12orf36
TCGA-OR-A5J1-01,44.547457,85.0144,97.5024,0.0,671.950,0.0,2428.43,462.056,0.0,385.687,...,0.0000,0.0,12.0077,0.0,17.2911,913.535,547.55,363.112,1.9212,0.0
TCGA-OR-A5J2-01,55.133642,165.8840,92.6050,0.0,812.911,0.0,5097.30,669.977,0.0,547.980,...,0.4026,0.0,0.8053,0.0,268.9890,303.978,1793.32,592.672,0.0000,0.0


In [15]:
y = np.array(cBio['Overall Survival (Months)'], dtype=float)
X = cBio.drop('Overall Survival (Months)', axis=1)

In [16]:
path = "/content/drive/MyDrive/gsoc data/PathwayCommons12.reactome.hgnc.sif.gz"  


#specify compression type because the file is compressed
df = pd.read_csv(path, sep="\t", compression="gzip", header=None, names=["Source", "InteractionType", "Target"])

In [17]:
df = df.drop(columns='InteractionType')

In [18]:
df.head()

,Source,Target
0,A1CF,APOBEC1
1,A1CF,APOBEC2
2,A1CF,APOBEC3A
3,A1CF,APOBEC3B
4,A1CF,APOBEC3C


In [19]:
all_nodes = sorted(list(set(df['Source']) | set(df['Target'])))
len(all_nodes)

12324

In [20]:
len(set(all_nodes) & set(X.columns))

9614

In [21]:
used_nodes = sorted(list(set(X.columns) & set(all_nodes)))
len(used_nodes)

9614

In [22]:
X = X.loc[:, used_nodes]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAC,AADAT,AAK1,AANAT,AARS,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0,10373.70,0.0,2225.26,1266.090,0.0000,121.0370,868.876,1.4409,3520.65,...,0.0,306.9160,0.0,2180.12,256.004,0.0000,253.602,183.958,146.974,596.062
TCGA-OR-A5J2-01,0.0,9844.91,0.0,1509.46,981.613,1.2079,31.0025,1627.030,0.0000,5470.94,...,0.0,99.4255,0.0,2553.48,104.684,0.4026,270.165,264.125,438.464,801.637


In [23]:
source_values = df['Source'].values
mask = np.isin(source_values, used_nodes)
df = df.loc[mask]

source_values = df['Target'].values
mask = np.isin(source_values, used_nodes)
df = df.loc[mask]
df = df.reset_index(drop=True)

In [24]:
len(set(df['Source']) | set(df['Target']))

9288

In [25]:
columns_to_select = sorted(set(df['Source']).union(set(df['Target'])))
X = X[columns_to_select]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAT,AAK1,AANAT,AARS,AARS2,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0,10373.70,0.0,2225.26,1266.090,121.0370,868.876,1.4409,3520.65,299.712,...,0.0,306.9160,0.0,2180.12,256.004,0.0000,253.602,183.958,146.974,596.062
TCGA-OR-A5J2-01,0.0,9844.91,0.0,1509.46,981.613,31.0025,1627.030,0.0000,5470.94,408.666,...,0.0,99.4255,0.0,2553.48,104.684,0.4026,270.165,264.125,438.464,801.637


In [26]:
source_nodes = df['Source'].tolist()
target_nodes = df['Target'].tolist()


# Create a dictionary to map each unique node to a unique index
node_to_index = {node: index for index, node in enumerate(X.columns)}

# Map the source and target nodes to their corresponding indices
source_indices = [node_to_index[node] for node in source_nodes]
target_indices = [node_to_index[node] for node in target_nodes]

In [27]:
# Convert the source and target indices to a PyTorch tensor
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

In [28]:
edge_index

tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])

In [29]:
edge_index.shape

torch.Size([2, 271771])

In [30]:
# Set the fixed index for splitting
split_index = int(0.8 * X.shape[0])

# Split the data based on the fixed index
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [31]:
X_train.shape, X_test.shape

((62, 9288), (16, 9288))

In [32]:
X_train = X_train.values
X_test = X_test.values

In [33]:
X_train[0]

array([    0.   , 10373.7  ,     0.   , ...,   183.958,   146.974,
         596.062])

In [34]:
X_test[1]

array([   0.    , 1728.03  ,    0.    , ...,  106.475 ,   39.4796,
        261.403 ])

In [35]:
y_train

array([ 44.54745701,  55.13364237,  68.74445212,  11.9998685 ,
        88.86477956,  16.1094125 ,  19.03540783,  44.44882796,
        30.31199658,  18.11486997,  57.53361607,  99.8783575 ,
        69.20472104,  66.24584936,  17.78610645,  76.63477661,
        16.1094125 ,  49.215899  ,  67.59377979,  18.47650985,
        39.25436434,  15.25462735,  36.2626163 , 121.2479863 ,
        12.59164283,  29.8188513 ,  66.50886018,  72.39372719,
        31.23253444,  18.14774633,  27.02436138,  33.82976625,
       108.1303219 ,  32.67909393, 113.9165598 ,  35.57221291,
        16.37242332,  49.0843936 ,  24.62438768,  11.3094651 ,
        46.48716179,  95.17703916, 153.6311931 , 127.4944932 ,
        68.25130684,  44.84334418,  12.85465365,   4.10954401,
       152.1517572 ,  31.79143242,  43.29815564,  28.30653911,
        29.09557156,  28.63530263,  23.63809712,  39.58312786,
         5.22733997,  39.35299339,  21.76414505,  52.24052339,
        78.41009962,  36.328369  ])

In [36]:
num_patients_train = X_train.shape[0]
num_patients_test = X_test.shape[0]

# Repeat the edge index for each patient in the training set
#edge_index_train_per_patient = edge_index.repeat(num_patients_train, 1)

# Repeat the edge index for each patient in the test set
#edge_index_test_per_patient = edge_index.repea(num_patients_test, 1)


# Convert the patient-specific edge indices to PyTorch tensors
#edge_index_train_per_patient = torch.tensor(edge_index_train_per_patient, dtype=torch.long)
#edge_index_test_per_patient = torch.tensor(edge_index_test_per_patient, dtype=torch.long)

# Create patient-specific graphs for the training set
graphs_train = []
for i in range(num_patients_train):
    node_features = X_train[i]  # Node features for the i-th patient
    target = y_train[i]  # Target label for the i-th patient
    graph_train = (node_features, edge_index, target)
    graphs_train.append(graph_train)

# Create patient-specific graphs for the test set
graphs_test = []
for i in range(num_patients_test):
    node_features = X_test[i]  # Node features for the i-th patient
    target = y_test[i]  # Target label for the i-th patient
    graph_test = (node_features, edge_index, target)
    graphs_test.append(graph_test)

In [37]:
# Check the number of patient-specific graphs
print(len(graphs_train))  # Should be 62
print(len(graphs_test))  # Should be 16

62
16


In [38]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 0 # Index of the patient
node_features, edge_index, y = graphs_train[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
print(y)

[    0.    10373.7       0.    ...   183.958   146.974   596.062]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
44.54745701


In [39]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 15  # Index of the patient
node_features, edge_index, y = graphs_test[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
print(y)

[1.20290e+00 1.17995e+04 0.00000e+00 ... 9.13745e+02 1.31836e+03
 6.15878e+02]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
42.50912319


In [49]:
# Convert graphs_train to a list of Data objects
data_train = [Data(x=torch.tensor(graph[0].reshape(len(graphs_train[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_train]

# Convert graphs_test to a list of Data objects
data_test = [Data(x=torch.tensor(graph[0].reshape(len(graphs_test[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_test]

In [50]:
graphs_train[0][0]

array([    0.   , 10373.7  ,     0.   , ...,   183.958,   146.974,
         596.062])

In [51]:
graphs_train[0]

(array([    0.   , 10373.7  ,     0.   , ...,   183.958,   146.974,
          596.062]),
 tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
         [ 451,  452,  453,  ..., 3323, 3340, 3341]]),
 44.54745701)

In [52]:
# Access the attributes of a specific data object in the training set
sample = data_train[0]  # Get the first data object
print(sample)  # Print the data object

# Access the node features, edge indices, and target label
node_features = sample.x
edge_index = sample.edge_index
target = sample.y

print(node_features)  # Print the node features
print(edge_index)  # Print the edge indices
print(target)  # Print the target label

Data(x=[9288, 1], edge_index=[2, 271771], y=44.54745701)
tensor([[    0.0000],
        [10373.7000],
        [    0.0000],
        ...,
        [  183.9580],
        [  146.9740],
        [  596.0620]], dtype=torch.float64)
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
tensor(44.5475, dtype=torch.float64)


In [54]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(data_train, batch_size=16, shuffle=True)
test_loader = DataLoader(data_test, batch_size=4, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Step 2:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Step 3:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Step 4:
Number of graphs in the current batch: 14
DataBatch(x=[130032, 1], edge_index=[2, 3804794], y=[14], batch=[130032], ptr=[15])



In [55]:
for step, data in enumerate(test_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 4
DataBatch(x=[37152, 1], edge_index=[2, 1087084], y=[4], batch=[37152], ptr=[5])

Step 2:
Number of graphs in the current batch: 4
DataBatch(x=[37152, 1], edge_index=[2, 1087084], y=[4], batch=[37152], ptr=[5])

Step 3:
Number of graphs in the current batch: 4
DataBatch(x=[37152, 1], edge_index=[2, 1087084], y=[4], batch=[37152], ptr=[5])

Step 4:
Number of graphs in the current batch: 4
DataBatch(x=[37152, 1], edge_index=[2, 1087084], y=[4], batch=[37152], ptr=[5])



In [61]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

In [70]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1)  # Regression output with 1 dimension

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final regression layer
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x.squeeze()  # Remove the extra dimension

In [82]:
model = GCN(hidden_channels=64, num_node_features=1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.double()  # Convert the model's parameters to Double type

num_epochs = 10  # Specify the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    total_loss = 0

    for step, data in enumerate(train_loader):
        optimizer.zero_grad()

        out = model(data.x.double(), data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1).double())
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / (step + 1)
    
    # Evaluate on the test set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        test_loss = 0
        for data in test_loader:
            out = model(data.x.double(), data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1).double())
            test_loss += loss.item()

        average_test_loss = test_loss / len(test_loader)
        #print(f"Epoch: {epoch+1:03d}, Test Loss: {average_test_loss:.4f}")
        print(f'Epoch: {epoch:03d}, Train loss: {average_loss:.4f}, Test Loss: {average_test_loss:.4f}')

Epoch: 000, Train loss: 17751.0125, Test Loss: 780.6482
Epoch: 001, Train loss: 14536.5825, Test Loss: 1400.9717
Epoch: 002, Train loss: 8672.4043, Test Loss: 757.9401
Epoch: 003, Train loss: 8233.0428, Test Loss: 1955.2005
Epoch: 004, Train loss: 6689.1689, Test Loss: 739.4835
Epoch: 005, Train loss: 5408.9922, Test Loss: 758.5592
Epoch: 006, Train loss: 4023.0989, Test Loss: 1054.2906
Epoch: 007, Train loss: 3333.1623, Test Loss: 1086.6984
Epoch: 008, Train loss: 2300.8816, Test Loss: 732.8917
Epoch: 009, Train loss: 2623.9930, Test Loss: 771.3651


In [83]:
model.eval()  # Set the model to evaluation mode

predictions = []  # List to store the predicted outputs

with torch.no_grad():
    for data in test_loader:
        out = model(data.x.double(), data.edge_index, data.batch)
        predictions.append(out.detach().numpy())  # Convert the predictions to NumPy array

# Concatenate the predictions from multiple batches
predictions = np.concatenate(predictions)

# Print the predictions
print(predictions)

[68.6032112  51.69334831 63.3760201  57.54936994 46.10642923 58.08948647
 60.26124462 49.75207209 50.67523135 50.36315221 51.00225132 61.97715144
 67.23728561 57.20699991 57.78701522 58.18229506]


In [84]:
model.eval()  # Set the model to evaluation mode
total_loss = 0

with torch.no_grad():
    for data in test_loader:
        out = model(data.x.double(), data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1, 1))
        total_loss += loss.item() * data.num_graphs

average_loss = total_loss / len(test_loader.dataset)
print(f"Average Loss on Test Data: {average_loss:.4f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Average Loss on Test Data: 852.0030
